In [ ]:
!wget https://datahack-prod.s3.amazonaws.com/train_file/train_LZdllcl.csv -O train.csv

In [ ]:
!wget https://datahack-prod.s3.amazonaws.com/test_file/test_2umaH9m.csv -O test.csv

In [ ]:
!wget https://datahack-prod.s3.amazonaws.com/sample_submission/sample_submission_M0L0uXE.csv -O sample_submission.csv

In [ ]:
import numpy as np
import pandas as pd 

#keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score


#visualisation
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
train = pd.read_csv('train.csv')
train.drop('employee_id',inplace=True,axis = 1)
train.head()

In [ ]:
train.dtypes

In [ ]:
train.isnull().sum()

In [ ]:
train.nunique()

In [ ]:
train['education'].fillna('other',inplace=True)

In [ ]:
train['previous_year_rating'].fillna(99,inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
train['is_promoted'].value_counts(normalize = True)

In [ ]:
train.shape

In [ ]:
y = train['is_promoted']
train = train.drop(['is_promoted'],axis = 1)

In [ ]:
train.head()

In [ ]:
train = pd.get_dummies(train)

In [ ]:
train.head()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train,y,test_size=0.15)

In [ ]:
print('Xtrain shape',X_train.shape)
print('Xvalid shape',X_valid.shape)
print('ytrain shape',y_train.shape)
print('yvalid shape',y_valid.shape)

In [ ]:
model = Sequential()

model.add(Dense(64, input_dim=59, kernel_initializer='normal', activation='relu'))

model.add(Dense(128,kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(Dense(256,kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(Dense(128,kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(Dense(64,kernel_initializer='normal', activation='relu'))

model.add(Dense(1, kernel_initializer='normal', activation='sigmoid')) 

In [ ]:
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)


In [ ]:
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)


In [ ]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)


In [ ]:
checkpointer = ModelCheckpoint(filepath='best_weights.hdf5', verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.00001,verbose = 1)
early_stopping = EarlyStopping(monitor='val_loss',min_delta=0.0001, patience=5,verbose=1)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])

In [ ]:
model.summary()

In [ ]:
model_info = model.fit(X_train,y_train,epochs=50,batch_size=32,validation_data=(X_valid,y_valid),verbose = 1,callbacks=[checkpointer,reduce_lr,early_stopping])